In [1]:
### Data handling imports
import pandas as pd
import numpy as np

import reverse_geocoder as rg
import unidecode

### Plotting imports
import matplotlib.pyplot as plt
import seaborn as sns
import copy

sns.set()
%matplotlib inline
%load_ext lab_black
import os
from collections import Counter
from collections import OrderedDict
from operator import itemgetter

import json

# Statistics imports
import scipy, scipy.stats

pd.set_option("display.max_columns", None)

import warnings

warnings.filterwarnings("ignore")


class color:
    BOLD = "\033[1m"
    UNDERLINE = "\033[4m"
    END = "\033[0m"

In [2]:
inputFolder = "../twitter-data-geo-output"
outputFolder = "./twitter-data-demographics-output"

In [3]:
def read_json(filepath):
    with open(filepath, "r") as file:
        return json.load(file)